In [1]:
import sklearn
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("file.csv")

In [3]:
test.tail()
#print(type(e))

,Unnamed: 0,tweet
996,996,What company is making the Biden administratio...
997,997,Wyoming can do so much better than this D.C. B...
998,998,The beatings will continue until morale improv...
999,999,Are Democrats saying it will no longer be a “W...
1000,1000,Crack pipes &gt; Hydroxychloroquine\n\nThanks...


In [4]:
sum(train["label"]==0)

30573

In [5]:
sum(train["label"]==1)

2390

In [6]:
!pip install tweet-preprocessor

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import re

#set up punctuations we want to be replaced
no_space= re.compile(
    "(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
need_space= re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")


In [8]:
import preprocessor as pre
def cleaning_data(data_frame):
    fin_out = []
    for line in data_frame:
        temp = pre.clean(line)
        temp = no_space.sub("",temp.lower())
        temp = no_space.sub(" ",temp)
        fin_out.append(temp)
    return fin_out

In [9]:

train_tweet = cleaning_data(train["tweet"])
train_tweet = pd.DataFrame(train_tweet)


In [10]:
train["clean_tweet"] = train_tweet
train.head(10)

,id,label,tweet,clean_tweet
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for credit i cant use cause they dont o...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,i love u take with u all the time in ur
4,5,0,factsguide: society now #motivation,factsguide society now
5,6,0,[2/2] huge fan fare and big talking before the...,2/2 huge fan fare and big talking before they ...
6,7,0,@user camping tomorrow @user @user @user @use...,camping tomorrow danny
7,8,0,the next school year is the year for exams.ð...,the next school year is the year for exams can...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,we won love the land
9,10,0,@user @user welcome here ! i'm it's so #gr...,welcome here im its so


In [11]:
test_tweet = cleaning_data(test["tweet"])
test_tweet = pd.DataFrame(test_tweet)
test["clean_tweet"] = test_tweet
test.tail(10)
#print(len(test_tweet))

,Unnamed: 0,tweet,clean_tweet
991,991,BREAKING: Donald Trump and the RNC funded SIGI...,breaking donald trump and the rnc funded sigin...
992,992,Remember folks you can choose where your hard ...,remember folks you can choose where your hard ...
993,993,All those people &amp; media sources trying to...,all those people &amp media sources trying to ...
994,994,OMG she misgendered (or gendered or who the he...,omg she misgendered or gendered or who the hel...
995,995,Wyoming you can do so much better than this D....,wyoming you can do so much better than this dc...
996,996,What company is making the Biden administratio...,what company is making the biden administratio...
997,997,Wyoming can do so much better than this D.C. B...,wyoming can do so much better than this dc bel...
998,998,The beatings will continue until morale improv...,the beatings will continue until morale improves
999,999,Are Democrats saying it will no longer be a “W...,are democrats saying it will no longer be a wi...
1000,1000,Crack pipes &gt; Hydroxychloroquine\n\nThanks...,crack pipes &gt hydroxychloroquinethanks democ...


In [12]:
from sklearn.model_selection import train_test_split
y = train.label.values
x_train, x_test, y_train, y_test = train_test_split(train.clean_tweet.values, y,
                                                    stratify=y,
                                                    random_state=1,
                                                    test_size=0.01, train_size=0.99, shuffle=True)
                                                    


In [13]:
print(train_tweet)

                                                       0
0      when a father is dysfunctional and is so selfi...
1      thanks for credit i cant use cause they dont o...
2                                    bihday your majesty
3                i love u take with u all the time in ur
4                                 factsguide society now
...                                                  ...
32958  we shouldnt be surprised given his actions ove...
32959  are you shitting me it was a garage pull it wa...
32960  how much money has the cheney family made from...
32961  breaking nancy pelosi just announced she wont ...
32962  as a self proclaimed honorary editor of it is ...

[32963 rows x 1 columns]


In [14]:
#print(list(x_train))
x_testf = np.array(test_tweet)
##   x_testf.append(x_test1[i])
print((x_testf))


[['bloodbath']
 ['a republican house has to vote to defund this nonsense at this point npr is nothing more than a taxpayer funded leftist super pac']
 ['maricopa county peeps get out and vote if youre in line when polls close they have to let you vote get out there and lets turn arizona red dont take any chances do it now']
 ...
 ['the beatings will continue until morale improves']
 ['are democrats saying it will no longer be a winter of deathwhat changed']
 ['crack pipes &gt hydroxychloroquinethanks democrats']]


In [15]:
x_test=x_testf.flatten()
print(x_test.shape)

(1001,)


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(binary=True, stop_words='english')
vectorizer.fit(list(x_train) + list(x_test))
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)


In [17]:
from sklearn import svm
svm = svm.SVC(kernel='linear', probability=True)
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)
y_pred_svm = svm.predict(x_test_vec)


In [19]:
output = open("output.txt","w")
for i in range(0,len(y_pred_svm)):
    if (y_pred_svm[i]==1):
        print("HATE FOUND", " ---> ", x_test[i], file = output) 
